In [82]:
import copy
from dataclasses import dataclass, field
from enum import IntEnum
from typing import TypeVar, Sequence, Optional, List, Iterator, Union

import clingo
import clingox.program


In [83]:
ForwardSymbol = TypeVar('ForwardSymbol', bound='Symbol')


class Symbol:

    def __neg__(self):
        return UnaryOperation(UnaryOperatorType.Minus, copy.deepcopy(self))

    def __add__(self, other):
        return BinaryOperation(copy.deepcopy(self), BinaryOperatorType.Plus, other)

    def is_function(self) -> bool:
        return isinstance(self, Function)

    def is_unary_operation(self) -> bool:
        return isinstance(self, UnaryOperation)

    def is_binary_operation(self) -> bool:
        return isinstance(self, BinaryOperation)

    def is_operation(self) -> bool:
        return self.is_unary_operation() or self.is_binary_operation()

    def is_variable(self) -> bool:
        return isinstance(self, Variable)

    def is_term(self) -> bool:
        return isinstance(self, Term)

    @classmethod
    def from_clingo_symbol(cls, symbol: clingo.Symbol) -> ForwardSymbol:
        if symbol.type is clingo.SymbolType.Function:
            name: str = symbol.name
            arguments = tuple(SubSymbol.from_clingo_symbol(argument) for argument in symbol.arguments)
            return Function(name, arguments)
        else:
            assert False, "Unknown clingo.SymbolType {}.".format(symbol.type)


class SubSymbol(Symbol):

    @classmethod
    def from_clingo_symbol(cls, symbol: clingo.Symbol) -> ForwardSymbol:
        if symbol.type is clingo.SymbolType.Number:
            return Term(IntegerConstant(symbol.number))
        elif symbol.type is clingo.SymbolType.String:
            return Term(StringConstant(symbol.string))
        elif symbol.type is clingo.SymbolType.Function:
            f_symbol = Symbol.from_clingo_symbol(symbol)
            if symbol.negative:
                return UnaryOperation(UnaryOperatorType.Minus, f_symbol)
            return f_symbol
        else:
            assert False, "Unknown clingo.SymbolType {}.".format(symbol.type)


In [84]:
@dataclass(frozen=True, order=True)
class Variable(SubSymbol):
    name: str

    def __str__(self):
        return self.name


@dataclass(frozen=True, order=True)
class StringConstant:
    string: str = ""

    def __str__(self):
        return '"{}"'.format(self.string)


@dataclass(frozen=True, order=True)
class IntegerConstant:
    number: int = 0

    def __str__(self):
        return str(self.number)


@dataclass(frozen=True, order=True)
class Term(SubSymbol):
    constant: Union[IntegerConstant, StringConstant] = field(default=IntegerConstant())

    def __str__(self):
        return str(self.constant)

In [85]:
ForwardAtom = TypeVar('ForwardAtom', bound='Atom')
ForwardFunction = TypeVar('ForwardFunction', bound='Function')
ForwardUnaryOperator = TypeVar('ForwardUnaryOperator', bound='UnaryOperator')


@dataclass(frozen=True, order=True)
class Function(Symbol):
    name: Optional[str] = None
    arguments: Sequence[SubSymbol] = ()

    @property
    def arity(self):
        return len(self.arguments)

    def __str__(self):
        if self.name is None and not self.arguments:
            return "()"
        elif self.name is not None and not self.arguments:
            return self.name
        elif self.name is None and self.arguments:
            return "({})".format(','.join(map(str, self.arguments)))
        else:
            return "{}({})".format(self.name, ','.join(map(str, self.arguments)))

    def match(self, name: Optional[str], arity: int = 0) -> bool:
        return name == self.name and arity == len(self.arguments)

    def match_signature(self, other: ForwardFunction) -> bool:
        return self.match(other.name, other.arity)


class UnaryOperatorType(IntEnum):
    Minus = 1


@dataclass(frozen=True, order=True)
class UnaryOperation(Symbol):
    operator: UnaryOperatorType
    argument: SubSymbol

    def __str__(self) -> str:
        if self.operator is UnaryOperatorType.Minus:
            if self.argument.is_operation():
                return "-({})".format(self.argument)
            return "-{}".format(self.argument)
        else:
            assert False, "Unknown UnaryOperatorType {}.".format(self.operator)


class BinaryOperatorType(IntEnum):
    Plus = 2


@dataclass(frozen=True, order=True)
class BinaryOperation(SubSymbol):
    left: SubSymbol
    operator: BinaryOperatorType
    right: SubSymbol

    def __str__(self) -> str:
        if self.operator is BinaryOperatorType.Plus:
            if self.left.is_operation() and self.right.is_operation():
                return "({})+({})".format(self.left, self.right)
            elif self.left.is_operation():
                return "({})+{}".format(self.left, self.right)
            elif self.right.is_operation():
                return "{}+({})".format(self.left, self.right)
            return "{}+{}".format(self.left, self.right)


In [86]:
@dataclass(frozen=True, order=True)
class Atom:
    symbol: Symbol = field(default_factory=Function)

    def __str__(self) -> str:
        return str(self.symbol)

    def is_function(self) -> bool:
        return isinstance(self.symbol, Function)

    def is_unary_operation(self) -> bool:
        return isinstance(self.symbol, UnaryOperation)

    def get_top_function(self) -> Function:
        current = self.symbol
        while not isinstance(current, Function):
            if isinstance(current, UnaryOperation):
                current = current.argument
            else:
                assert False, "Unknown Type {} for Symbol {}.".format(type(current).__name__, current)
        return current

    def is_isomorph_to(self, other: ForwardAtom) -> bool:
        assert isinstance(other, Atom), "Atom {} should have type {}, but has type {}.".format(other, Atom.__name__,
                                                                                               type(other).__name__)
        stack_self: List[SubSymbol] = [self.symbol]
        stack_other: List[SubSymbol] = [other.symbol]
        while stack_self:
            if len(stack_self) != len(stack_other):
                return False
            current_self = stack_self.pop()
            current_other = stack_other.pop()
            if current_self.is_variable() and current_other.is_variable():
                if current_self.name != current_other.name:
                    return False
            elif not current_self.is_variable() and not current_other.is_variable():
                if type(current_self) != type(current_other):
                    return False
                if current_self.is_unary_operation() and current_other.is_unary_operation():
                    if current_self.operator != current_other.operator:
                        return False
                    stack_self.append(current_self.argument)
                    stack_other.append(current_other.argument)
                elif current_self.is_term() and current_other.is_term():
                    if current_self != current_other:
                        return False
                elif current_self.is_function() and current_other.is_function():
                    if not current_self.match_signature(current_other):
                        return False
                    stack_self.extend(current_self.arguments)
                    stack_other.extend(current_other.arguments)

        return True

    @staticmethod
    def from_clingo_symbol(symbol: clingo.Symbol) -> ForwardAtom:
        assert symbol.type is clingo.SymbolType.Function, "clingo.Symbol {} should have type {}, but has type {}.".format(
            symbol, clingo.SymbolType.Function, symbol.type)
        return Atom(Symbol.from_clingo_symbol(symbol))




In [87]:
class Sign(IntEnum):
    NoSign = 0
    DefaultNeg = 1

In [88]:
@dataclass(frozen=True, order=True)
class Literal:
    atom: Atom = field(default_factory=Atom)
    sign: Sign = Sign.NoSign

    def is_neg(self):
        return self.sign is Sign.DefaultNeg

    def is_pos(self):
        return self.sign is Sign.NoSign

    def __str__(self):
        if self.sign is Sign.DefaultNeg:
            return "not {}".format(self.atom)
        return str(self.atom)

    def __abs__(self):
        return Literal(sign=Sign.NoSign, atom=copy.deepcopy(self.atom))

    def __neg__(self):
        return Literal(sign=Sign(self.sign ^ 1), atom=self.atom)

    def __invert__(self):
        return Literal(sign=Sign(self.sign ^ 1), atom=self.atom)

In [89]:
class RuleLike:
    def is_rule(self) -> bool:
        return isinstance(self, Rule)

    def is_external(self) -> bool:
        return isinstance(self, External)

In [90]:

ForwardRule = TypeVar('ForwardRule', bound='Rule')
ForwardExternal = TypeVar('ForwardExternal', bound='External')

class RuleType(IntEnum):
    Disjunctive = 0
    Choice = 1

@dataclass(frozen=True, order=True)
class Rule(RuleLike):
    head: Sequence[Literal] = ()
    body: Sequence[Literal] = ()
    rule_type: RuleType = RuleType.Disjunctive

    def __str__(self) -> str:
        if self.head is None and not self.body:
            return ":-."
        elif self.head is None:
            return ":- {}.".format(', '.join(map(str, self.body)))
        elif not self.body:
            return "{}.".format(self.head)
        else:
            return "{} :- {}.".format(self.head, ', '.join(map(str, self.body)))

    def is_ground(self) -> bool:
        pass

    def is_fact(self) -> bool:
        return self.head is not None and not self.body

    def as_external(self) -> ForwardExternal:
        return External(self.head.atom[0], self.body, ExternalType.false)

    def is_constraint(self) -> bool:
        return self.head is None

    def is_normal_rule(self) -> bool:
        return self.head is not None and self.body

    def is_head_relevant(self, atom: Atom) -> bool:
        if self.head is None:
            return False
        #if self.head.atom == atom:
        #    return True
        return self.head.atom.is_isomorph_to(atom)


In [91]:
ForwardExternalType = TypeVar('ForwardExternalType', bound='ExternalType')


class ExternalType(IntEnum):
    false = 0
    true = 1
    free = 2

    @staticmethod
    def from_truth_value(tv: clingo.TruthValue) -> ForwardExternalType:
        if tv is clingo.TruthValue.False_:
            return ExternalType.false
        elif tv is clingo.TruthValue.True_:
            return ExternalType.true
        else:
            assert tv is clingo.TruthValue.Free
            return ExternalType.free


@dataclass(frozen=True, order=True)
class External(RuleLike):
    atom: Atom = field(default_factory=Atom)
    body: Sequence[Literal] = ()
    external_type: ExternalType = ExternalType.false

    def __str__(self):
        if not self.body:
            return "#external {}. [{}]".format(self.atom, self.external_type.name)
        return "#external {} : {}. [{}]".format(self.atom, ', '.join(map(str, self.body)), self.external_type.name)


In [92]:
ForwardProgram = TypeVar('ForwardProgram', bound='Program')


@dataclass(frozen=True, order=True)
class Program:
    part: str = field(default='base')
    parameters: Sequence[Function] = field(default_factory=tuple)
    rules: Sequence[RuleLike] = field(default_factory=tuple)

    def __str__(self):
        if not self.parameters:
            return "#program {}. {}".format(self.part, ' '.join(map(str, self.rules)))
        return "#program {}({}). {}".format(self.part, ','.join(map(str, self.parameters)),
                                            ' '.join(map(str, self.rules)))

    def facts(self) -> Iterator[Atom]:
        for rule in self.rules:
            if rule.is_rule():
                assert isinstance(rule, Rule)
                if rule.is_fact():
                    yield rule
            if rule.is_external():
                assert isinstance(rule, External)
                if not rule.body:
                    yield rule

    def facts_as_external(self) -> ForwardProgram:
        new_rules = []
        for rule in self.rules:
            if rule.is_fact():
                new_rules.append(rule.as_external())
            else:
                new_rules.append(rule)
        return Program(new_rules)

    def support_rules(self, atom: Atom) -> Iterator[RuleLike]:
        for rule in self.rules:
            if rule.is_rule():
                assert isinstance(rule, Rule)
                if rule.is_head_relevant(atom):
                    yield rule
            elif rule.is_external():
                assert isinstance(rule, External)
                if rule.atom.is_isomorph_to(atom):
                    yield rule

    def ground(self, ctl: Optional[clingo.Control] = None, parts=(('base', ()),)) -> ForwardProgram:
        if ctl is None:
            ctl = clingo.Control()
        prg = clingox.program.Program()
        obs = clingox.program.ProgramObserver(prg)
        ctl.register_observer(obs)
        ctl.add('base', [], str(self))
        ctl.ground(parts)
        new_rules = []
        for fact in prg.facts:
            new_rules.append(Rule(Literal(Atom.from_clingo_symbol(fact.symbol))))
        for rule in prg.rules:
            if len(rule.head) == 0:
                head = None
            elif len(rule.head) == 1:
                if rule.head[0] not in prg.output_atoms:
                    continue
                head = Literal(Atom.from_clingo_symbol(prg.output_atoms[rule.head[0]]))
            else:
                assert False, "Unexpected length of rule head"
            body = tuple(
                Literal(sign=Sign(body_literal < 0), atom=Atom.from_clingo_symbol(prg.output_atoms[abs(body_literal)]))
                for body_literal in rule.body)
            new_rules.append(Rule(head, body))
        for external in prg.externals:
            new_rules.append(External(Atom.from_clingo_symbol(prg.output_atoms[external.atom]), (),
                                      ExternalType.from_truth_value(external.value)))
        return Program(rules=new_rules)

    def evaluate_forwards(self, ctl: Optional[clingo.Control] = None, parts=(('base', ()),)) -> Iterator[
        Sequence[Atom]]:
        if ctl is None:
            ctl = clingo.Control()
        ctl.configuration.solve.models = 0
        ctl.add('base', [], str(self))
        ctl.ground(parts)
        with ctl.solve(yield_=True) as solve_handle:
            for model in solve_handle:
                symbols = sorted(model.symbols(shown=True))
                atoms = tuple(Atom.from_clingo_symbol(symbol) for symbol in symbols)
                yield atoms

    def cautious_consequences(self) -> Sequence[Atom]:
        ctl = clingo.Control()
        ctl.configuration.solve.models = 0
        ctl.configuration.solve.enum_mode = 'cautious'
        ctl.add('base', [], str(self))
        ctl.ground([('base', [])])
        with ctl.solve(yield_=True) as solve_handle:
            model = None
            for m in solve_handle:
                model = m
            symbols = sorted(model.symbols(shown=True))
            atoms = tuple(Atom.from_clingo_symbol(symbol) for symbol in symbols)
            return atoms


def ground(programs: Sequence[Program], ctl: Optional[clingo.Control], parts=(('base', ()),)):
    if ctl is None:
        ctl = clingo.Control()
    prg = clingox.program.Program()
    obs = clingox.program.ProgramObserver(prg)
    ctl.register_observer(obs)
    ctl.add('base', [], '\n'.join(map(str, programs)))
    ctl.ground(parts)
    new_rules = []
    for fact in prg.facts:
        new_rules.append(Rule(Literal(Atom.from_clingo_symbol(fact.symbol))))
    for rule in prg.rules:
        if len(rule.head) == 0:
            head = None
        elif len(rule.head) == 1:
            if rule.head[0] not in prg.output_atoms:
                continue
            head = Literal(Atom.from_clingo_symbol(prg.output_atoms[rule.head[0]]))
        else:
            assert False, "Unexpected length of rule head"
        body = tuple(
            Literal(sign=Sign(body_literal < 0), atom=Atom.from_clingo_symbol(prg.output_atoms[abs(body_literal)]))
            for body_literal in rule.body)
        new_rules.append(Rule(head, body))
    for external in prg.externals:
        new_rules.append(External(Atom.from_clingo_symbol(prg.output_atoms[external.atom]), (),
                                  ExternalType.from_truth_value(external.value)))
    return Program(rules=new_rules)


def evaluate_forwards(programs: Sequence[Program],
                      ctl: Optional[clingo.Control] = None,
                      parts=(('base', ()),)) -> Iterator[Sequence[Atom]]:
    if ctl is None:
        ctl = clingo.Control()
    ctl.configuration.solve.models = 0
    ctl.add('base', [], '\n'.join(map(str, programs)))
    ctl.ground(parts)
    with ctl.solve(yield_=True) as solve_handle:
        for model in solve_handle:
            symbols = sorted(model.symbols(shown=True))
            atoms = tuple(Atom.from_clingo_symbol(symbol) for symbol in symbols)
            yield atoms


In [93]:
program = """
% F
fluent(m).
fluent(ab).
% D
default(ab).
% E
action(d).
% AD
impossible(d, m, -ab).
% Aleph
occ_at(d, 1).
"""
m = Function('m')
ab = Function('ab')
d = Function('d')
p_raw_rules = (
    Rule(Literal(Atom(Function('fluent', (m,))))),
    Rule(Literal(Atom(Function('fluent', (ab,))))),
    Rule(Literal(Atom(Function('default', (ab,))))),
    Rule(Literal(Atom(Function('action', (d,))))),
    Rule(Literal(Atom(Function('impossible', (d, m, -ab))))),
    Rule(Literal(Atom(Function('occ_at', (d, Term(IntegerConstant(1))))))),
)
p_raw = Program(rules=p_raw_rules)
print(p_raw)

#program base. fluent(m). fluent(ab). default(ab). action(d). impossible(d,m,-ab). occ_at(d,1).


In [94]:
__t = Function('__t')

In [95]:
signature = []
new_rules = []
for rule in p_raw.rules:
    if rule.head.atom.symbol.name == 'impossible':
        impossible_if_stm: Function = rule.head.atom.symbol
        new_rules.append(
            Rule(None, (Literal(Atom(Function('occ_at', (impossible_if_stm.arguments[0], __t)))),
                        *(Literal(Atom(Function('obs_at', (argument, __t)))) for argument in
                          impossible_if_stm.arguments[1:]))))
    elif rule.head.atom.symbol.name == 'if':
        if_stm: Function = rule.head.atom.symbol
        head = Literal(Atom(Function('obs_at', (if_stm.arguments[0], __t))))
        body = tuple(
            Literal(Atom(Function('obs_at', (argument, __t)))) for argument in if_stm.arguments[1:])
        new_rules.append(Rule(head, body))
    elif rule.head.atom.symbol.name == 'causes':
        causes_stm: Function = rule.head.atom.symbol
        head = Literal(Atom(Function('obs_at', (
            causes_stm.arguments[1], __t + Term(IntegerConstant(1))))))
        body = (Literal(Atom(Function('occ_at', (causes_stm.arguments[0], __t)))),
                *(Literal(Atom(Function('obs_at', (argument, __t)))) for argument in
                  causes_stm.arguments[2:]))
        new_rules.append(Rule(head, body))
    else:
        signature.append(rule)

new_rules.append(
    Rule(
        Literal(Atom(Function('obs_at', (
            Variable('F'), __t + Term(IntegerConstant(1)))))),
        (
            Literal(Atom(Function('obs_at', (Variable('F'), __t)))),
            -Literal(Atom(Function('obs_at', (-Variable('F'), __t + Term(IntegerConstant(1)))))),
            -Literal(Atom(Function('obs_at', (Function('u', (Variable('F'),)), __t + Term(IntegerConstant(1)))))),
        ),
    )
)

new_rules.append(
    Rule(
        Literal(Atom(Function('obs_at', (-Variable('F'), __t + Term(IntegerConstant(1)))))),
        (
            -Literal(Atom(Function('obs_at', (Variable('F'), __t + Term(IntegerConstant(1)))))),
            -Literal(Atom(Function('obs_at', (Function('u', (Variable('F'),)), __t + Term(IntegerConstant(1)))))),
            Literal(Atom(Function('obs_at', (-Variable('F'), __t)))),
        )
    )
)

new_rules.append(
    Rule(
        Literal(Atom(Function('obs_at', (Function('u', (Variable('F'),)), __t + Term(IntegerConstant(1)))))),
        (
            -Literal(Atom(Function('obs_at', (Variable('F'), __t + Term(IntegerConstant(1)))))),
            Literal(Atom(Function('obs_at', (Function('u', (Variable('F'),)), __t)))),
            -Literal(Atom(Function('obs_at', (-Variable('F'), __t + Term(IntegerConstant(1)))))),
        )
    )
)
new_rules.append(
    Rule(
        Literal(Atom(Function('obs_at', (Variable('F'), Term(IntegerConstant(1)))))),
        (
            Literal(Atom(Function('init', (Variable('F'),)))),
        )
    )
)
new_rules.append(
    Rule(
        Literal(Atom(Function('obs_at', (-Variable('F'), Term(IntegerConstant(1)))))),
        (
            Literal(Atom(-Function('init', (Variable('F'),)))),
        )
    )
)
new_rules.append(
    Rule(
        Literal(Atom(Function('obs_at', (Variable('F'), Term(IntegerConstant(1)))))),
        (
            Literal(Atom(Function('forced', (Variable('F'),)))),
            Literal(Atom(Function('default', (Variable('F'),)))),
            -Literal(Atom(-Function('init', (Variable('F'),)))),
        )
    )
)

p = Program(rules=signature)
pa = Program(part='action_language', parameters=(__t,), rules=new_rules)
print(p, pa, sep='\n')

#program base. fluent(m). fluent(ab). default(ab). action(d). occ_at(d,1).
#program action_language(__t). :- occ_at(d,__t), obs_at(m,__t), obs_at(-ab,__t). obs_at(F,__t+1) :- obs_at(F,__t), not obs_at(-F,__t+1), not obs_at(u(F),__t+1). obs_at(-F,__t+1) :- not obs_at(F,__t+1), not obs_at(u(F),__t+1), obs_at(-F,__t). obs_at(u(F),__t+1) :- not obs_at(F,__t+1), obs_at(u(F),__t), not obs_at(-F,__t+1). obs_at(F,1) :- init(F). obs_at(-F,1) :- -init(F). obs_at(F,1) :- forced(F), default(F), not -init(F).
